In [2]:
from workflow.scripts.utils import read_list_input_paths, model_levels_to_pressure_levels
from pyclim_noresm.general_util_funcs import global_avg
import xarray as xr
import xesmf
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from workflow.scripts.plotting_tools import create_facet_plot
import geocat.comp as geocomp
from matplotlib import ticker

In [3]:
paths = snakemake.input.paths

dsets, vname = read_list_input_paths(paths)

In [4]:
for m, ds in dsets.items():
    print(m)
    dsets[m] = model_levels_to_pressure_levels(ds).compute()

In [5]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()
gl = ax.gridlines()
gl.bottom_labels = True
# p = plt.Rectangle((-10,5),50,30)
ax.add_patch(mpatches.Rectangle([-18,3],width=50,height=33,
                                facecolor='red',
                                    alpha=0.2,
                                transform = ccrs.PlateCarree()))
ax.set_global()

In [6]:
# fig, axes, cax = create_facet_plot(len(dsets), subplot_kw={})
fig, ax = plt.subplots()
for m, ds in dsets.items():
    ds = ds.sel(lon=slice(-18,32), lat=slice(3,36)).copy()
#     ds = ds.mean(dim='time',skipna=True)
    ds = global_avg(ds)
    ds = ds.mean(dim='time',skipna=True)
    ds = ds.assign_coords(plev=ds['plev']/100)
    ax.plot(ds['loaddust']*1000, ds.plev, label=m)
    ax.invert_yaxis()
    ax.set_ylim(1050,150)
    ax.legend()
    ax.set_xscale('log')
    ax.set_xlim(1e-4, 10)
    ax.set_yscale('log')
    locator = ticker.FixedLocator([1000,900,800,700,600,500,400,300,200,100])
    ax.yaxis.set_major_locator(locator)
    formatter = ticker.FixedFormatter([1000,900,800,700,600,500,400,300,200,100])
    ax.yaxis.set_major_formatter(formatter)
#     ds['loaddust'].plot(ax=ax)
    ax.set_xlabel('Dust loading $\mathrm{g\; m}^{-2}$')
    ax.set_ylabel('Pressure [hPa]')
    ax.set_title('Vertical distribution of Sharan dust')
plt.savefig(snakemake.output.path, bbox_inches='tight', dpi=144)